In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc

In [ ]:
# young = np.loadtxt("/Users/bbrener1/battle/other_sc/young_counts.txt")
# old = np.loadtxt("/Users/bbrener1/battle/other_sc/old_counts.txt")
header = np.loadtxt("/Users/bbrener1/battle/other_sc/header.txt",dtype=str)
young = sc.read_text("/Users/bbrener1/battle/other_sc/young_counts.txt")
old = sc.read_text("/Users/bbrener1/battle/other_sc/old_counts.txt")

young.var_names = header
old.var_names = header

In [ ]:
sc.pp.neighbors(young)
sc.tl.louvain(young)
sc.tl.umap(young)
sc.pl.umap(young,color='louvain')


In [ ]:
mean_sort = np.argsort(np.mean(young.X,axis=0))
mean_sort.shape

header[mean_sort[-100:]]

In [ ]:
print(young.shape)
print(old.shape)
print(header.shape)

In [ ]:
import sys
# sys.path.append('/localscratch/bbrener1/rusty_forest_v3/src')
sys.path.append('../src')
import tree_reader as tr 
import lumberjack

In [ ]:
forest = lumberjack.fit(
    young.X,
    header=header,
    trees=100,
    braids=1,
    ifs=1500,
    ofs=1500,
    ss=500,
    depth=3,
    leaves=10,
    sfr=.5,
    reduce_input="true",
)

forest.set_cache(True)

forest.arguments

In [ ]:
# forest.backup("./scanpy_cmp_aging_blood")
# forest.tsne_coordinates = young.obsm['X_umap']

In [ ]:
forest = tr.Forest.load('scanpy_cmp_aging_blood')
forest.arguments

In [ ]:
forest.interpret_splits(mode='sister',metric="sister",k=20,relatives=False,pca=False)

In [ ]:
forest.maximum_spanning_tree(mode='samples')

In [ ]:
forest.tsne_coordinates = young.obsm["X_umap"]
forest.html_tree_summary(n=10)

In [ ]:
young_prediction = forest.predict(young.X)
old_prediction = forest.predict(old.X)

In [ ]:
young_features, young_samples = young_prediction.prediction_report(mode="additive_mean")
old_features, old_samples = old_prediction.prediction_report(mode="additive_mean")

In [ ]:
delta,match = young_prediction.compare_predictions(old_prediction)

In [ ]:
delta_sort = np.argsort(np.abs(delta))
delta[delta_sort]

In [ ]:
old_prediction.feature_mse()

In [ ]:
feature_mask = np.random.random(young.X.shape[1]) > .5
in_counts,out_counts = young.X.T[feature_mask].T,young.X.T[~feature_mask].T

In [ ]:
from sklearn.decomposition import PCA

model = PCA(n_components=30).fit(out_counts)
transformed = model.transform(out_counts)
recovered = model.inverse_transform(transformed)

centered = out_counts - np.mean(out_counts,axis=0)
transformed_residual = np.power(centered,2)

recovered_residual = np.power(out_counts - recovered,2)

pca_recovered_per_sample = np.sum(recovered_residual,axis=1)
pca_recovered_fraction_per_sample = np.sum(recovered_residual,axis=1) / np.sum(transformed_residual,axis=1)
print(np.sum(transformed_residual))
print(np.sum(recovered_residual))

print(f"Remaining variance:{(np.sum(recovered_residual) / np.sum(transformed_residual))}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor as RF

forest_model = RF(n_estimators=100,min_samples_split=10,verbose=4,n_jobs=10,max_samples=200).fit(in_counts,out_counts)


In [ ]:
recovered = forest_model.predict(in_counts)

recovered_residual = np.power(out_counts - recovered,2)

print(f"Remaining variance:{(np.sum(recovered_residual) / np.sum(transformed_residual))}")

In [ ]:
plt.figure()
plt.scatter(*young.obsm['X_umap'].T,c=young_samples)
plt.colorbar()

In [ ]:
dir(young_prediction)

In [ ]:
np.mean(young.X,axis=0)[2201]

In [ ]:
list(header).index("Znf683")

In [ ]:
for gene in header:
    if 'X' in gene:
        print(gene)
#     if 'Tr' in gene:
#         print(gene)